# 介绍
本指南的目的是向你展示如何正确地对 TGI 进行基准测试。如需更多背景信息和解释，请先查看这篇[热门博客](https://huggingface.co/blog/tgi-benchmarking)。

## 设置
确保你有一个已安装 TGI 的环境；Docker 是一个很好的选择。这里的命令可以很容易地复制/粘贴到终端中，这可能更为便捷。无需强制使用 Jupyter。如果你只是想测试一下，可以复制并使用[derek-thomas/tgi-benchmark-space](https://huggingface.co/spaces/derek-thomas/tgi-benchmark-space)。

# TGI 启动器

In [1]:
!text-generation-launcher --version

text-generation-launcher 2.2.1-dev0


以下是 TGI 的不同设置选项，请确保通读并决定哪些设置对你的使用场景最为重要。

以下是一些最重要的设置：
- `--model-id` 
- `--quantize` 量化可以节省内存，但并不总是能提高速度
- `--max-input-tokens` 这可以让 TGI 优化预填充操作
- `--max-total-tokens` 与上述设置结合，TGI 现在知道最大输入和输出 token 的限制
- `--max-batch-size` 这个设置让 TGI 知道它每次可以处理多少个请求。

最后这三个设置共同提供了必要的限制，以便为你的使用场景进行优化。通过合理设置这些选项，你可以显著提高性能。

In [2]:
!text-generation-launcher -h

Text Generation Launcher

Usage: text-generation-launcher [OPTIONS]

Options:
      --model-id <MODEL_ID>
          The name of the model to load. Can be a MODEL_ID as listed on <https://hf.co/models> like `gpt2` or `OpenAssistant/oasst-sft-1-pythia-12b`. Or it can be a local directory containing the necessary files as saved by `save_pretrained(...)` methods of transformers [env: MODEL_ID=] [default: bigscience/bloom-560m]
      --revision <REVISION>
          The actual revision of the model if you're referring to a model on the hub. You can use a specific commit id or a branch like `refs/pr/2` [env: REVISION=]
      --validation-workers <VALIDATION_WORKERS>
          The number of tokenizer workers used for payload validation and truncation inside the router [env: VALIDATION_WORKERS=] [default: 2]
      --sharded <SHARDED>
          Whether to shard the model across multiple GPUs By default text-generation-inference will use all available GPUs to run the model. Setting it to `false` 

我们可以直接从本指南中启动，因为我们不需要命令是交互式的。

在本指南中，我们将使用默认设置，因为目的是理解基准测试工具。

如果你在 Space 上运行，以下参数已被更改，因为我们不希望与 Spaces 服务器发生冲突：
- `--hostname`
- `--port`

根据你的需求，你可以随意更改或删除这些参数。

In [3]:
!RUST_BACKTRACE=1 \
text-generation-launcher \
--model-id astronomer/Llama-3-8B-Instruct-GPTQ-8-Bit \
--quantize gptq \
--hostname 0.0.0.0 \
--port 1337

2024-08-16T12:07:56.411768Z  INFO text_generation_launcher: Args {
    model_id: "astronomer/Llama-3-8B-Instruct-GPTQ-8-Bit",
    revision: None,
    validation_workers: 2,
    sharded: None,
    num_shard: None,
    quantize: Some(
        Gptq,
    ),
    speculate: None,
    dtype: None,
    trust_remote_code: false,
    max_concurrent_requests: 128,
    max_best_of: 2,
    max_stop_sequences: 4,
    max_top_n_tokens: 5,
    max_input_tokens: None,
    max_input_length: None,
    max_total_tokens: None,
    waiting_served_ratio: 0.3,
    max_batch_prefill_tokens: None,
    max_batch_total_tokens: None,
    max_waiting_tokens: 20,
    max_batch_size: None,
    cuda_graphs: None,
    hostname: "0.0.0.0",
    port: 1337,
    shard_uds_path: "/tmp/text-generation-server",
    master_addr: "localhost",
    master_port: 29500,
    huggingface_hub_cache: None,
    weights_cache_override: None,
    disable_custom_kernels: false,
    cuda_memory_fraction: 1.0,
    rope_scaling: None,
    rop

# TGI 基准测试
现在让我们学习如何启动基准测试工具！

在这里，我们可以看到 TGI 基准测试的不同设置选项。

以下是一些更重要的 TGI 基准测试设置：

- `--tokenizer-name` 这是必需的，以便工具知道使用哪个分词器
- `--batch-size` 这对于负载测试非常重要。我们应该使用足够的值来查看吞吐量和延迟的变化。请注意，在基准测试工具中，batch-size 是虚拟用户的数量。
- `--sequence-length` 也就是输入 tokens，这对于匹配你的使用场景需求非常重要
- `--decode-length` 也就是输出 tokens，这对于匹配你的使用场景需求非常重要
- `--runs` 默认值是 10

<blockquote style="border-left: 5px solid #80CBC4; background: #263238; color: #CFD8DC; padding: 0.5em 1em; margin: 1em 0;">
  <strong>💡 提示：</strong> 当你在探索时，使用较小的 <code style="background: #37474F; color: #FFFFFF; padding: 2px 4px; border-radius: 4px;">--runs</code> 值，但在最终确定时使用较高的值，以获得更精确的统计数据。
</blockquote>

In [4]:
!text-generation-benchmark -h

Text Generation Benchmarking tool

Usage: text-generation-benchmark [OPTIONS] --tokenizer-name <TOKENIZER_NAME>

Options:
  -t, --tokenizer-name <TOKENIZER_NAME>
          The name of the tokenizer (as in model_id on the huggingface hub, or local path) [env: TOKENIZER_NAME=]
      --revision <REVISION>
          The revision to use for the tokenizer if on the hub [env: REVISION=] [default: main]
  -b, --batch-size <BATCH_SIZE>
          The various batch sizes to benchmark for, the idea is to get enough batching to start seeing increased latency, this usually means you're moving from memory bound (usual as BS=1) to compute bound, and this is a sweet spot for the maximum batch size for the model under test
  -s, --sequence-length <SEQUENCE_LENGTH>
          This is the initial prompt sent to the text-generation-server length in token. Longer prompt will slow down the benchmark. Usually the latency grows somewhat linearly with this for the prefill step [env: SEQUENCE_LENGTH=] [default: 1

这是一个示例命令。请注意，我多次添加了感兴趣的 batch sizes，以确保所有的值都能被基准测试工具使用。我还根据预计的用户活动，考虑了哪些 batch sizes 是重要的。

<blockquote style="border-left: 5px solid #FFAB91; background: #37474F; color: #FFCCBC; padding: 0.5em 1em; margin: 1em 0;">
  <strong>⚠️ 警告：</strong> 请注意，TGI 基准测试工具是设计用来在终端中运行的，而不是在 Jupyter 笔记本中。这意味着你需要将命令复制/粘贴到 Jupyter 终端标签中。我在这里放置它是为了方便。
</blockquote>

In [ ]:
!text-generation-benchmark \
--tokenizer-name astronomer/Llama-3-8B-Instruct-GPTQ-8-Bit \
--sequence-length 70 \
--decode-length 50 \
--batch-size 1 \
--batch-size 2 \
--batch-size 4 \
--batch-size 8 \
--batch-size 16 \
--batch-size 32 \
--batch-size 64 \
--batch-size 128 